In [14]:
import qiskit
import numpy as np

import json, glob, math

In [8]:
glob.glob("*")

['curves.nb', 'elephant QFT.ipynb', 'README.md']

In [39]:
def qubits_necessary(n):
    """log2 of n"""
    l2 = math.log2(n)
    assert l2 % 1 == 0, "not an integer power of 2"
    return int(l2)

def normalized(arr):
    """l2-normalization"""
    return arr / np.linalg.norm(arr, 2)

## Data Loading

In [54]:
with open("./Resources/elephant.3.json") as f:
    data = json.load(f)[:-1]
    data = normalized(np.array([complex(a, b) for a, b in data]))
    data = np.round(data, 2)

In [55]:
data

array([-0.34-0.2j , -0.35+0.03j, -0.37+0.21j,  0.25+0.2j ,  0.25+0.34j,
        0.31+0.04j,  0.2 -0.23j, -0.07-0.25j])

## Classical FT

In [57]:
data_ft = np.fft.fft(data)

## State Loading

In [40]:
from qiskit import QuantumCircuit

In [59]:
# number of qubits necessary
qubit_count = qubits_necessary(len(data_ft))
print(f"need {qubit_count} qubits")

# array of complex amplitudes
desired_state = normalized(data_ft)

# initialization circut
circuit = QuantumCircuit(qubit_count, qubit_count, name="data_loader_trivial")
circuit.initialize(desired_state, range(qubit_count))

need 3 qubits


## Inverse QFT